In [4]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from pathlib import Path
import seaborn as sns

sys.path.append(os.path.abspath(os.path.join('../src')))
from constant_drift_diffusion import *
from accumulators import *

# bayesflow
sys.path.append(os.path.abspath(os.path.join('../../BayesFlow')))
from bayesflow.networks import InvariantNetwork, InvertibleNetwork
from bayesflow.amortizers import SingleModelAmortizer
from bayesflow.trainers import ParameterEstimationTrainer
from bayesflow.diagnostics import *

from tensorflow.keras.layers import Dense, GRU, LSTM, Conv1D, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.utils.np_utils import to_categorical

In [ ]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

In [5]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Constants

In [6]:
# simulation 
N_SIM = 500
N_OBS = 100

# bayesflow
PARAM_NAMES = ["a", "ndt", "bias", "kappa"]
N_PARAMS = len(PARAM_NAMES)
N_EPOCHS = 50
ITER_PER_EPOCH = 1000
BATCH_SIZE = 32
N_SAMPLES = 200

## Simulator Test

In [7]:
n_obs = 100
a     = 3.0
ndt   = 0.2
bias  = 0.5
kappa = 5
theta = np.array([a, ndt, bias, kappa])

unique_motions = np.array([-0.725, -0.675, -0.625, -0.575, -0.525, 0.525,  0.575,  0.625,  0.675,  0.725], dtype=np.float32)
amplitude = np.repeat(unique_motions, 10)
condition = to_categorical(pd.factorize(amplitude)[0])

rt, resp = const_dm_simulator(theta, 1, amplitude)

In [9]:
p_, x_ = const_dm_batch_simulator(32, 100)
x_.shape

(32, 100, 12)

## Bayes Flow

In [10]:
sum_meta = {
    'n_dense_s1': 2,
    'n_dense_s2': 3,
    'n_dense_s3': 2,
    'n_equiv':    2,
    'dense_s1_args': {'activation': 'relu', 'units': 32},
    'dense_s2_args': {'activation': 'relu', 'units': 64},
    'dense_s3_args': {'activation': 'relu', 'units': 32},
}

summary_net = InvariantNetwork()

2021-12-06 17:31:14.102872: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# invertable inference network
meta_dict={
    'n_coupling_layers': 5,
    's_args': {
        'units': [128, 128],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    't_args': {
        'units': [128, 128],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    'alpha': 1.9,
    'permute': True,
    'use_act_norm': True,
    'n_params': N_PARAMS
}

inference_net = InvertibleNetwork(meta_dict)

# Connect summary and inference network
amortizer = SingleModelAmortizer(inference_net, summary_net)

In [12]:
# Learning-rate decay
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0005, 1000, 0.99, staircase=True
)

trainer = ParameterEstimationTrainer(
    network=amortizer, 
    generative_model=const_dm_batch_simulator,
    learning_rate=learning_rate,
    checkpoint_path='../src/selected_checkpoints/const_dm',
    clip_value=3,
    max_to_keep=5
)

Initializing networks from scratch.


In [ ]:
%%time
# online training
losses = trainer.train_online(N_EPOCHS, ITER_PER_EPOCH, BATCH_SIZE, n_obs=N_OBS)

## Paramter Recovery

In [ ]:
# Simulate and amortized inference
p_, x_ = const_dm_batch_simulator(n_sim=N_SIM,n_obs=N_OBS)
samples = amortizer.sample(x_, n_samples=N_SAMPLES)
param_means = samples.mean(axis=1)

In [ ]:
# Recovery plot
true_vs_estimated(theta_true=p_, theta_est=param_means,
                  param_names=PARAM_NAMES, dpi=300, figsize=(20,6),font_size=16)

In [ ]:
sns.pairplot(pd.DataFrame(samples[0], columns=PARAM_NAMES))